In [12]:
# Load preprocessed Data
import sys
sys.path.append('/home/alexabades/recsys')
import pandas as pd 
import numpy as np 
import torch

In [2]:
data_path = '../data/raw/frappe/frappe.csv'
df = pd.read_csv(data_path, sep='\t')

In [3]:
df.head()

,user,item,cnt,daytime,weekday,isweekend,homework,cost,weather,country,city
0,0,0,1,morning,sunday,weekend,unknown,free,sunny,United States,0
1,1,1,7,afternoon,saturday,weekend,unknown,free,cloudy,Spain,0
2,2,2,6,evening,monday,workday,unknown,free,cloudy,Spain,369
3,3,3,1,sunset,thursday,workday,unknown,free,unknown,United States,1028
4,4,4,428,night,thursday,workday,home,free,sunny,Switzerland,147


- 0 - user
- 1 - item
- 2 - rating
- 3 - cnt
- 4 - daytime_afternoon
- 5 - daytime_evening
- 6 - daytime_morning
- 7 - daytime_night
- 8 - daytime_noon
- 9 - daytime_sunrise
- 10 - daytime_sunset
- 11 - weather_sunny
- 12 - weather_cloudy
- 13 - weather_unknown
- 14 - weather_foggy
- 15 - weather_rainy
- 16 - weather_stormy
- 17 - weather_drizzle
- 18 - weather_snowy
- 19 - isweekend_weekend
- 20 - isweekend_workday
- 21 - homework_home
- 22 - homework_unknown
- 23 - homework_work

We have to train the enconder with the frappe dataset. We can use the entire data set, we have to ensure the that the data has the same columns, but we do not need to remove the users/items with less interactions from the data set, or do we?
- Try with both. With the train dataset and the test dataset that we will use for the RS. We'll be lossing aprox a 13% of the total interactions. At the end the data that will not be used in the RS could not be good for the AE.
Additionally, is the leave one out enogh to test the performance ? 
- Use the entire dataset to train


In [ ]:
ctx_categorical_columns = [
    "daytime",
    "weather",
    "isweekend",
    "homework",
]

# Autoencoder dummy data no sparse

21 

In [7]:
df = pd.read_csv('../../../data/processed/FRAPPE/CNCF/frapppe5Context/frapppe5Context.train.rating', sep='\t', header=None)

In [ ]:
def create_noise_tensor(context_input):
    noise_tensor = torch.randint(
        0, 2, size=context_input.shape, dtype=torch.float32, device=_device
    )
    noise = torch.rand(context_input.shape[0], 1)
    noise_tensor[:, 0] = noise.squeeze()
    return noise_tensor

In [10]:
df_context = df.iloc[:, 3:].copy()

In [14]:
torch.randint(0, 2, size=df_context.shape, dtype=torch.float32).shape

torch.Size([83731, 21])

In [15]:
torch.rand(df_context.shape[0], 1)

tensor([[0.8244],
        [0.7759],
        [0.4316],
        ...,
        [0.6369],
        [0.9476],
        [0.2737]])

In [71]:
data = np.ones((df_context.shape[0]+654, df.shape[1]))
# Select at most 2 columns per each row and replace it with 0
for i in range(data.shape[0]):
    idx = np.random.choice(data.shape[1], 2, replace=False)
    data[i, idx] = 0
cnt = torch.rand(df_context.shape[0]+654, 1)
# mimic user ID
data[:, 0] = list(range(data.shape[0]))
# mimic item ID
data[:, 1] = range(data.shape[0])
# update 2nd column with the context
data[:, 3] = cnt.squeeze().numpy()

In [73]:
data = pd.DataFrame(data, columns=df.columns)


In [75]:
# train test split
from sklearn.model_selection import train_test_split


In [76]:
train, test = train_test_split(data, test_size=0.2)

In [79]:
train.to_csv('../../../data/processed/FRAPPE/AE/frappe5Context/frapppe5Context.train.rating', sep='\t', index=False, header=False)
test.to_csv('../../../data/processed/FRAPPE/AE/frappe5Context/frapppe5Context.test.rating', sep='\t', index=False, header=False)

In [27]:
# data.to_csv('../../../data/processed/FRAPPE/AE/frappe5Context/dummy_no_sparse.csv', sep='\t', index=False, header=False)
